In [1]:
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
try:
    sc = SparkContext('local', 'Pyspark demo')
except ValueError:
    print('SparkContext already exists!')

try:
    spark = SparkSession.builder.appName('Recommendation_system').getOrCreate()
except ValueError:
    print('SparkSession already exists!')

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/12 22:25:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
path = '../resources/dataset_credit_risk.csv'
with open(path) as f:
    data = f.readlines()
data.__len__()

777716

In [4]:
import sys, math
n_slices = math.ceil(sys.getsizeof(data) / 81920)
n_slices

80

In [5]:
spark_df = spark.read.csv(sc.parallelize(data, numSlices=n_slices), header=True)

22/02/12 22:25:37 WARN TaskSetManager: Stage 0 contains a task of very large size (1689 KiB). The maximum recommended task size is 1000 KiB.
22/02/12 22:25:39 WARN TaskSetManager: Stage 1 contains a task of very large size (1689 KiB). The maximum recommended task size is 1000 KiB.


+-------+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+--------------------+---------------+------+----------+--------------+----------+------------------+
|loan_id|     id|code_gender|flag_own_car|flag_own_realty|cnt_children|amt_income_total|    name_income_type| name_education_type|  name_family_status|name_housing_type|days_birth|days_employed|flag_mobil|flag_work_phone|flag_phone|flag_email|     occupation_type|cnt_fam_members|status|  birthday|job_start_date| loan_date|       loan_amount|
+-------+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+--------------------+---------------+------+----------+--------------

In [ ]:
spark_df = spark_df.drop(
 'loan_id',
 'code_gender',
 'flag_own_realty',
 'cnt_children',
 'amt_income_total',
 'name_income_type',
 'name_education_type',
 'name_family_status',
 'name_housing_type',
 'days_birth',
 'days_employed',
 'flag_mobil',
 'flag_work_phone',
 'flag_phone',
 'flag_email',
 'occupation_type',
 'cnt_fam_members',
 'status')

**Feature nb_previous_loans**

In [7]:
from pyspark.sql.window import Window

window_spec  = Window.partitionBy("id").orderBy("loan_date")
spark_df = spark_df.withColumn("nb_previous_loans",F.row_number().over(window_spec))
spark_df = spark_df.withColumn('nb_previous_loans', F.col('nb_previous_loans')-1)


**Feature avg_amount_loans_previous**

In [15]:
%%time
# compute every element of a group by clause and later return the mean value iterate of each row increasing the elements 
# calculated on the mean by one row in each iteration
#window = Window.orderBy("id").rowsBetween(Window.unboundedPreceding, Window.currentRow)
window = Window.orderBy("id").partitionBy("id").rowsBetween(Window.unboundedPreceding, -1)
spark_df = spark_df.withColumn("avg_amount_loans_previous", F.mean('loan_amount').over(window))

22/02/12 22:34:59 WARN TaskSetManager: Stage 29 contains a task of very large size (1689 KiB). The maximum recommended task size is 1000 KiB.


+-------+------------+----------+--------------+----------+------------------+-----------------+-------------------------+---+
|     id|flag_own_car|  birthday|job_start_date| loan_date|       loan_amount|nb_previous_loans|avg_amount_loans_previous|age|
+-------+------------+----------+--------------+----------+------------------+-----------------+-------------------------+---+
|5008804|           Y|1988-11-04|    2009-04-11|2019-02-01|102.28336090329137|                0|                     null| 33|
|5008804|           Y|1988-11-04|    2009-04-11|2019-02-15|136.60204916713766|                1|       102.28336090329137| 33|
|5008804|           Y|1988-11-04|    2009-04-11|2019-02-17|114.73369384225273|                2|       119.44270503521452| 33|
|5008804|           Y|1988-11-04|    2009-04-11|2019-05-20|103.53905003353589|                3|        117.8730346375606| 33|
|5008804|           Y|1988-11-04|    2009-04-11|2019-07-05|112.94814667072303|                4|       114.2895

**Feature age**

In [17]:
%%time
spark_df = spark_df.withColumn("age",F.floor(F.datediff(F.current_date(),F.col("birthday"))/F.lit(365)))

22/02/12 22:36:16 WARN TaskSetManager: Stage 35 contains a task of very large size (1689 KiB). The maximum recommended task size is 1000 KiB.


+-------+------------+----------+--------------+----------+------------------+-----------------+-------------------------+---+
|     id|flag_own_car|  birthday|job_start_date| loan_date|       loan_amount|nb_previous_loans|avg_amount_loans_previous|age|
+-------+------------+----------+--------------+----------+------------------+-----------------+-------------------------+---+
|5008804|           Y|1988-11-04|    2009-04-11|2019-02-01|102.28336090329137|                0|                     null| 33|
|5008804|           Y|1988-11-04|    2009-04-11|2019-02-15|136.60204916713766|                1|       102.28336090329137| 33|
|5008804|           Y|1988-11-04|    2009-04-11|2019-02-17|114.73369384225273|                2|       119.44270503521452| 33|
|5008804|           Y|1988-11-04|    2009-04-11|2019-05-20|103.53905003353589|                3|        117.8730346375606| 33|
|5008804|           Y|1988-11-04|    2009-04-11|2019-07-05|112.94814667072303|                4|       114.2895

**Feature years_on_the_job**

In [18]:
%%time
spark_df = spark_df.withColumn("years_on_the_job",F.floor(F.datediff(F.current_date(),F.col("job_start_date"))/F.lit(365)))


CPU times: user 2.91 ms, sys: 660 µs, total: 3.57 ms
Wall time: 107 ms


**Feature flag_own_car**

In [19]:
%%
spark_df = spark_df.withColumn('flag_own_car', F.when(spark_df['flag_own_car'] == 'N', 0).otherwise(1))

In [21]:
%%time
spark_df.filter("id = 5150479").select("*").show(20)

22/02/12 22:37:11 WARN TaskSetManager: Stage 38 contains a task of very large size (1689 KiB). The maximum recommended task size is 1000 KiB.


+-------+------------+----------+--------------+----------+------------------+-----------------+-------------------------+---+----------------+
|     id|flag_own_car|  birthday|job_start_date| loan_date|       loan_amount|nb_previous_loans|avg_amount_loans_previous|age|years_on_the_job|
+-------+------------+----------+--------------+----------+------------------+-----------------+-------------------------+---+----------------+
|5150479|           0|1987-09-22|    2021-05-12|2019-03-01|102.30940021783978|                0|                     null| 34|               0|
|5150479|           0|1987-09-22|    2021-05-12|2019-03-21|112.63284909681813|                1|       102.30940021783978| 34|               0|
|5150479|           0|1987-09-22|    2021-05-12|2019-05-03| 139.5850258127131|                2|       107.47112465732896| 34|               0|
|5150479|           0|1987-09-22|    2021-05-12|2019-05-19|126.35760048140841|                3|       118.17575837579034| 34|          

In [22]:
df_processed = spark_df[['id', 'age', 'years_on_the_job', 'nb_previous_loans', 'avg_amount_loans_previous', 'flag_own_car']].toPandas()
df_processed

22/02/12 22:37:22 WARN TaskSetManager: Stage 41 contains a task of very large size (1689 KiB). The maximum recommended task size is 1000 KiB.


,id,age,years_on_the_job,nb_previous_loans,avg_amount_loans_previous,flag_own_car
0,5008804,33,12,0,NaN,1
1,5008804,33,12,1,102.283361,1
2,5008804,33,12,2,119.442705,1
3,5008804,33,12,3,117.873035,1
4,5008804,33,12,4,114.289538,1
...,...,...,...,...,...,...
777710,5150487,53,6,25,132.585287,1
777711,5150487,53,6,26,132.016323,1
777712,5150487,53,6,27,131.044545,1
777713,5150487,53,6,28,130.375785,1


In [24]:
df_processed.tail(30)

,id,age,years_on_the_job,nb_previous_loans,avg_amount_loans_previous,flag_own_car
777685,5150487,53,6,0,NaN,1
777686,5150487,53,6,1,180.240014,1
777687,5150487,53,6,2,142.768625,1
777688,5150487,53,6,3,142.266333,1
777689,5150487,53,6,4,132.696773,1
777690,5150487,53,6,5,132.678782,1
777691,5150487,53,6,6,128.287698,1
777692,5150487,53,6,7,125.588518,1
777693,5150487,53,6,8,127.939567,1
777694,5150487,53,6,9,135.007000,1


In [2]:
df = pd.read_csv('../resources/dataset_credit_risk.csv')

In [3]:
df.to_parquet('../resources/dataset_credit_risk.parquet')